# Pandas: эффект(ив)ный анализ таблиц

Давайте познакомимся с **Pandas** чуть подробнее. Напомним, что NumPy замечательно подходит для работы с численными массивами, однако, астрономические данные часто преставлены в виде структурированных таблиц: звездные каталоги, логи наблюдений, экзопланетные базы данных и прочее. Эти таблицы содержат в себе данные различных типов в разных колонках (текстовые, численные, дата и время), а также заголовки для колонок и строк.

Pandas является мощной библиотекой, созданной специально для анализа таких структуированных табличных данных. Его основная структура, датафрейм (**DataFrame**), является чем-то вроде мощной электронной таблицы, доступной прямо из кода в Python. 

Этот ноутбук покажет вам, как взаимодействовать с датафреймами, благодаря чему их очистка, анализ и исследования станут более прозрачными и эффективными.

**Задачи:**

*   Изучить основную структуру данных Pandas: датафрейм.
*   Создать датафрейм различными способами.
*   Выбрать отдельные колонки и строки в таблице.
*   Отфильтровать датафрейм согласно условиям.
*   Обработать колонки Dataframe базовыми операциями.
*   Считать данные из локального файла (CSV).
*   Считать данные напрямую из онлайн источников (URLs).
*   Научиться обрабатывать отсутствующие значения.

**Ноутбук предполагает знания основного синтаксиса Python (переменные, типы данных, списки, циклы) и массивов NumPy.**

**Замечание**
Запустите ячейку ниже по `SHIFT+ENTER` для загрузки и установки нужных пакетов, если вы не уверены, что такие установлены. Если они уже есть, её можно пропустить.

In [33]:
!pip install numpy pandas matplotlib

## Что такое DataFrame

Пусть у нас есть следующая таблиац с данными о галактиках:

```code
Name                   | Type      | Distance (light-years)
---                    | ---       | ---
Andromeda              | Spiral    | 2.537 million
Milky Way              | Spiral    | 0 (our galaxy)
Triangulum             | Spiral    | 3.0 million
Large Magellanic Cloud | Irregular | 0.16 million
```

В Python, можно представить эти данные в виде списка из списков или в виде списка словарей:

In [34]:
galaxies_list = [
    ['Andromeda', 'Spiral', 2.537],
    ['Milky Way', 'Spiral', 0.0],
    ['Triangulum', 'Spiral', 3.0],
    ['Large Magellanic Cloud', 'Irregular', 0.16]
]

galaxies_list_of_dicts = [
    {'Name': 'Andromeda', 'Type': 'Spiral', 'Distance_Mly': 2.537},
    {'Name': 'Milky Way', 'Type': 'Spiral', 'Distance_Mly': 0.0},
    {'Name': 'Triangulum', 'Type': 'Spiral', 'Distance_Mly': 3.0},
    {'Name': 'Large Magellanic Cloud', 'Type': 'Irregular', 'Distance_Mly': 0.16}
]

Метод рабочий, однако, анализировать данные в таком виде достаточно трудно. Необходимо будет использовать вложенные циклы и сложные условные конструкции.

Pandas предоставляет объект типа `DataFrame`, который подходит для такого представления гораздо лучше:

In [35]:
import pandas as pd # стандартное сокращение pandas

# Создадим датафрейм из списка словарей:
galaxies_df = pd.DataFrame(galaxies_list_of_dicts)

# Выведем его средствами Jupyter Notebook:
print("Our Galaxies DataFrame:")
galaxies_df

Our Galaxies DataFrame:


,Name,Type,Distance_Mly
0,Andromeda,Spiral,2.537
1,Milky Way,Spiral,0.000
2,Triangulum,Spiral,3.000
3,Large Magellanic Cloud,Irregular,0.160


Обратите внимание, что структура колонок и строк сохранилась. Каждый ключ в словаре стал названием колонки, плюс у каждой строки появился номер-индекс.

## Создание датафреймов

Наиболее часто встречаемый способ создания датафреймов:

1.  **Из словаря:** ключи в словаре - названия колонок, значения - списки или массивы NumPy с данными для колонок. Подходит для небольших датасетов, определяемых внутри кода.
2.  **Из списка словарей:** каждый словарь представляет собой строку.
3.  **Из файла:** считываем данные из CSV, Excel, JSON-ов и других типов файлов.
4.  **Из интернета:** считываем таблицу непосредственно по ссылке.

Давайте посмотрим на методы создания из словарей, затем перейдем к файлам и ссылкам.

### Пример 1: создание датафреймов из словаря

Снова создадим таблицу со звездами:

In [36]:
star_data_dict = {
    'Name': ['Sirius', 'Canopus', 'Alpha Centauri', 'Arcturus'],
    'Spectral_Type': ['A1V', 'F0', 'G2V', 'K1.5III'],
    'Apparent_Magnitude': [-1.46, -0.72, -0.27, -0.04]
}

# создаем датафрейм из словаря, в качестве индексов сделаем английские числительные
bright_stars_df = pd.DataFrame(star_data_dict, index=['First', 'Second', 'Third', 'Fourth'])

print("Датафрейм из словарей:")
bright_stars_df

Датафрейм из словарей:


,Name,Spectral_Type,Apparent_Magnitude
First,Sirius,A1V,-1.46
Second,Canopus,F0,-0.72
Third,Alpha Centauri,G2V,-0.27
Fourth,Arcturus,K1.5III,-0.04


Каждый ключ из словаря стал заголовком колонки, список стал самой колонкой, и к каждой строке добавился индекс, который мы указали как опциональный аргумент index.

## Доступ к данным

Для обращения к конкретным колонкам, строкам или значениям используется индексация:

*   **Выбор колонок:** происходит по заголовку колонки, аналогично обращению к ключу в словаре. Можно выделить несколько колонок
*   **Выбор строк:** строки можно выбирать по их индексному значению (`.loc`) или целочисленной позиции (`.iloc`).
*   **Выбор конкретной ячейки:** комбинация выбора строки и столбца.


### Пример 2: выделение колонок

Используем bright_stars_df из предыдущего примера:

In [37]:
# выбор одной колонки
# результатом является объект Series (одномерный именованный массив)
star_names = bright_stars_df['Name'] # аналогично выбору ключа в словаре
print("\nВыбираем колонку 'Name':")
star_names


Выбираем колонку 'Name':


First             Sirius
Second           Canopus
Third     Alpha Centauri
Fourth          Arcturus
Name: Name, dtype: object

In [38]:
# Выбор нескольких колонок
# результатом является датафрейм.
names_and_magnitudes = bright_stars_df[['Name', 'Apparent_Magnitude']] # список названий колонок!
names_and_magnitudes

,Name,Apparent_Magnitude
First,Sirius,-1.46
Second,Canopus,-0.72
Third,Alpha Centauri,-0.27
Fourth,Arcturus,-0.04


### Пример 3: выбор строк через .loc и .iloc

Снова используем тот же датафрейм bright_stars_df:

In [39]:
# Обращаемся к строке по заголовку строки, используя .loc
# Заголовки по-умолчанию - это целые числа от 0 (0, 1, 2, ...)
# Мы специально изменили заголовки на (First, Second...) для отличия
sirius_row = bright_stars_df.loc['First'] # достаем первую строку
print("Вытаскиваем строку с заголовком First (Sirius), используя .loc:")
sirius_row

Вытаскиваем строку с заголовком First (Sirius), используя .loc:


Name                  Sirius
Spectral_Type            A1V
Apparent_Magnitude     -1.46
Name: First, dtype: object

In [40]:
# По целочисленному индексу строку можно вытащить с помощью .iloc
# Всё аналогично индексации списков или массивов NumPy
canopus_row = bright_stars_df.iloc[1]
print("Вытаскиваем строку под индексом 1 (Canopus), используя .iloc:")
canopus_row

Вытаскиваем строку под индексом 1 (Canopus), используя .iloc:


Name                  Canopus
Spectral_Type              F0
Apparent_Magnitude      -0.72
Name: Second, dtype: object

In [41]:
# выбор нескольких строк делается с помощью срезов и .loc
first_two_stars_loc = bright_stars_df.loc['First':'Second'] # 'Second' здесь тоже включается, в отличие от обычных срезов!
print("Вытаскиваем первые две строки с помощью .loc['First':'Second']:")
print(first_two_stars_loc)

Вытаскиваем первые две строки с помощью .loc['First':'Second']:
           Name Spectral_Type  Apparent_Magnitude
First    Sirius           A1V               -1.46
Second  Canopus            F0               -0.72


In [42]:
# выбор нескольких строк через .iloc
first_two_stars_iloc = bright_stars_df.iloc[0:2] # а тут 2 уже не включается, как обычно
print("\nВытаскиваем первые две строки с помощью .iloc[0:2]:")
print(first_two_stars_iloc)


Вытаскиваем первые две строки с помощью .iloc[0:2]:
           Name Spectral_Type  Apparent_Magnitude
First    Sirius           A1V               -1.46
Second  Canopus            F0               -0.72


### Пример 4: выбор отдельных ячеек в таблице

In [43]:
# Снова тот же датафрейм bright_stars_df
print("Исходный датафрейм:")
print(bright_stars_df)

# Выбираем звездную величину Сириуса через .loc[заголовок_строки, заголовок_столбца]
sirius_mag = bright_stars_df.loc['First', 'Apparent_Magnitude']
print("\nЗначение ячейки через .loc['First', 'Apparent_Magnitude']:", sirius_mag)
print(f"Тип ячейки: {type(sirius_mag)}") # вернулся численный тип

Исходный датафрейм:
                  Name Spectral_Type  Apparent_Magnitude
First           Sirius           A1V               -1.46
Second         Canopus            F0               -0.72
Third   Alpha Centauri           G2V               -0.27
Fourth        Arcturus       K1.5III               -0.04

Значение ячейки через .loc['First', 'Apparent_Magnitude']: -1.46
Тип ячейки: <class 'numpy.float64'>


In [44]:
# Можно обратиться к ячейке по её целочисленным индексам с помощью .iloc[индекс_строки, индекс_столбца]
canopus_spectral_type = bright_stars_df.iloc[1, 1]
print("Спектральный тип Канопуса (через .iloc[1, 1]):", canopus_spectral_type)

# Срезами можно вытащить часть таблицы:
some_data = bright_stars_df.loc['Second':'Third', 'Spectral_Type':'Apparent_Magnitude':]
print("Часть таблицы:")
some_data

Спектральный тип Канопуса (через .iloc[1, 1]): F0
Часть таблицы:


,Spectral_Type,Apparent_Magnitude
Second,F0,-0.72
Third,G2V,-0.27


In [45]:
# аналогично через индексы
some_data = bright_stars_df.iloc[0:2, 0:2]
print("Часть таблицы:")
some_data

Часть таблицы:


,Name,Spectral_Type
First,Sirius,A1V
Second,Canopus,F0


## Фильтрация датафрейма

Частая задача - выделить определенные строки из таблицы, которые удовлетворяют заданному условию. С помощью булевых индексов, как в NumPy, это сделать достаточн просто. Нужно создать условие, которое выведет Серию из значений `True`/`False`, и затем использовать её в качестве индекса:

### Пример 5: фильтрация

In [46]:
# снова мучаем bright_stars_df
print("Исходный датафрейм:")
print(bright_stars_df)

# Условие 1: оставим звезды ярче -0.5 
is_brighter_than_05 = bright_stars_df['Apparent_Magnitude'] < -0.5
print("\nБулева серия для ярких звезд:")
print(is_brighter_than_05)

Исходный датафрейм:
                  Name Spectral_Type  Apparent_Magnitude
First           Sirius           A1V               -1.46
Second         Canopus            F0               -0.72
Third   Alpha Centauri           G2V               -0.27
Fourth        Arcturus       K1.5III               -0.04

Булева серия для ярких звезд:
First      True
Second     True
Third     False
Fourth    False
Name: Apparent_Magnitude, dtype: bool


In [67]:
# Используем эту серию в качестве индекса
very_bright_stars_df = bright_stars_df[is_brighter_than_05]
print("Отфильтрованный датафрейм только со звездами ярче -0.5:")
print(very_bright_stars_df)

# Условие 2: Найдем звезды главной последовательности (с 'V' в названии)
is_spectral_type_V = bright_stars_df['Spectral_Type'].str.contains('V')
print("\nБулева серия для условия на 'V':")
print(is_spectral_type_V)

Отфильтрованный датафрейм только со звездами ярче -0.5:
           Name Spectral_Type  Apparent_Magnitude  Distance_pc  \
First    Sirius           A1V               -1.46          2.6   
Second  Canopus            F0               -0.72         99.0   

        Absolute_Magnitude  
First             1.465133  
Second           -5.698176  

Булева серия для условия на 'V':
First      True
Second    False
Third      True
Fourth    False
Name: Spectral_Type, dtype: bool


In [68]:
# Аналогичная фильтрация без посредника-переменной
V_stars_df = bright_stars_df[bright_stars_df['Spectral_Type'].str.contains('V')]
print("Отфильтрованные значения с 'V':")
V_stars_df

Отфильтрованные значения с 'V':


,Name,Spectral_Type,Apparent_Magnitude,Distance_pc,Absolute_Magnitude
First,Sirius,A1V,-1.46,2.6,1.465133
Third,Alpha Centauri,G2V,-0.27,1.3,4.160283


Несколько условий можно сочетать следующими логическими операторами:

```code
&  (AND)
|  (OR)
~  (NOT)
```

**ВАЖНО:** каждое условие нужно обрамить в скобки `()`


### Пример 6: фильтрация несколькими условиями

In [49]:
# Найдем звезды ярче -0.5 величины со спектральным типом 'A1V'
bright_and_A1V = (bright_stars_df['Apparent_Magnitude'] < -0.5) & (bright_stars_df['Spectral_Type'] == 'A1V')
filtered_df = bright_stars_df[bright_and_A1V]
print("Звезды ярче -0.5 с типом 'A1V':")
filtered_df

Звезды ярче -0.5 с типом 'A1V':


,Name,Spectral_Type,Apparent_Magnitude
First,Sirius,A1V,-1.46


## Основные операции и создание новых колонок

Как и в случае массивов NumPy, колонки Pandas (Series) можно использовать в качестве аргументов математических операций. С их помощью можно легко создавать и добавлять новые колонки в уже существующую таблицу.

### Пример 7: операции и создание колонок

In [50]:
# используем наш многострадальный датафрейм
print("Звездочки:")
bright_stars_df

Звездочки:


,Name,Spectral_Type,Apparent_Magnitude
First,Sirius,A1V,-1.46
Second,Canopus,F0,-0.72
Third,Alpha Centauri,G2V,-0.27
Fourth,Arcturus,K1.5III,-0.04


Добавим колонку для абсолюной звездной величины звезд:

`Абсолютная величина (M) = Видимая величина (m) - 5 * log10(расстояние в парсеках / 10)`

Давайте добавим колонку с расстояниями в парсеках, создав новый объект типа `Series`:

In [54]:
distances_pc = pd.Series([2.6, 99.0, 1.3, 11.4], index=['First', 'Second', 'Third', 'Fourth']) # раз мы изменили заголовки строк, они у нас должны совпадать
bright_stars_df['Distance_pc'] = distances_pc # присваивание похоже на создание нового ключа в словаре

In [55]:
# Теперь подсчитаем абсолютную величину
# Вытащим из numpy векторизованный десятичный логарифм log10
from numpy import log10

bright_stars_df['Absolute_Magnitude'] = bright_stars_df['Apparent_Magnitude'] - 5 * log10(bright_stars_df['Distance_pc'] / 10)

print("Датафрейм с добавленными колонками 'Distance_pc' и 'Absolute_Magnitude':")
bright_stars_df

Датафрейм с добавленными колонками 'Distance_pc' и 'Absolute_Magnitude':


,Name,Spectral_Type,Apparent_Magnitude,Distance_pc,Absolute_Magnitude
First,Sirius,A1V,-1.46,2.6,1.465133
Second,Canopus,F0,-0.72,99.0,-5.698176
Third,Alpha Centauri,G2V,-0.27,1.3,4.160283
Fourth,Arcturus,K1.5III,-0.04,11.4,-0.324524


In [60]:
# Как и в NumPy, в Pandas есть встроенные общестатистические функции вроде mean, max, min, sum и так далее:
average_apparent_mag = bright_stars_df['Apparent_Magnitude'].mean()
print(f"Средняя видимая величина: {average_apparent_mag:.2f}")

max_absolute_mag = bright_stars_df['Absolute_Magnitude'].max()
print(f"Максимальная абсолютная величина: {max_absolute_mag:.2f}")

Средняя видимая величина: -0.62
Максимальная абсолютная величина: 4.16


## Чтение из файлов (CSV)

Для чтения текстовых данных в Pandas есть функция `read_csv()` для загрузки этих файлов напрямую в DataFrame.

### Пример 8: читаем локальный .csv

Давайте создадим тестовый текстовый файл `exoplanet_data.csv` вручную или с помощью следующего кода:

In [61]:
csv_content = """Name,Radius_Earth,OrbitalPeriod_days,HasWater
Kepler-186f,1.11,129.9,True
Kepler-1649b,1.06,8.68,False
TRAPPIST-1e,0.91,6.10,True
51 Pegasi b,1.50,4.23,False
"""
with open("exoplanet_data.csv", "w") as f:
    f.write(csv_content)

print("Создал тестовый файл 'exoplanet_data.csv'")

Создал тестовый файл 'exoplanet_data.csv'


In [63]:
# считаем csv в DataFrame:
exoplanets_df = pd.read_csv("exoplanet_data.csv")

print("Датафрейм из 'exoplanet_data.csv':")
print(exoplanets_df)

# посмотрим, какие типы колонок предположил Pandas
print("\nКраткая информация про датафрейм:")
exoplanets_df.info()

Датафрейм из 'exoplanet_data.csv':
           Name  Radius_Earth  OrbitalPeriod_days  HasWater
0   Kepler-186f          1.11              129.90      True
1  Kepler-1649b          1.06                8.68     False
2   TRAPPIST-1e          0.91                6.10      True
3   51 Pegasi b          1.50                4.23     False

Краткая информация про датафрейм:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                4 non-null      object 
 1   Radius_Earth        4 non-null      float64
 2   OrbitalPeriod_days  4 non-null      float64
 3   HasWater            4 non-null      bool   
dtypes: bool(1), float64(2), object(1)
memory usage: 232.0+ bytes


## Чтение CSV по ссылке

`read_csv()` может принимать в качестве аргумента ссылку на таблицу в сети. Pandas автоматически ее загрузит и превратит в датафрейм.

### Пример 9: чтение CSV из сети (упрощенные данные экзопланетного архива)

Ниже приведена ссылка на упрощенную таблицу данных архива экзопланет NASA, часто используемого в туториалах. *(внимание: ссылка может не сработать)*

In [64]:
exoplanet_url = "https://raw.githubusercontent.com/astroryan97/Exploring-Exoplanet-Data/main/open_exoplanet_catalogue.csv"

try:
    # читаем прямо этот URL
    online_exoplanets_df = pd.read_csv(exoplanet_url)

    print("Прочитанный из URL датафрейм:")
    print(online_exoplanets_df.head()) # вывести только первые 5 строк (голову)
    print(f"\nПолное число строк в таблице: {len(online_exoplanets_df)}") # len делает то, что ожидается

    # Быстрая проверка - сколько из этих экзопланет содержат 'KOI' в названии?
    koi_planets = online_exoplanets_df[online_exoplanets_df['name'].str.contains('KOI', na=False)] # строки с отсутсвующими значениями не учитываем
    print(f"\nЧисло планет с KOI в названии: {len(koi_planets)}")


except Exception as e:
    print(f"Не получается считать URL. Убедитесь, что он доступен.")
    print(f"Детали ошибки: {e}")

Прочитанный из URL датафрейм:
           name  binaryflag    mass  radius     period  semimajoraxis  \
0   KOI-1843.03           0  0.0014   0.054   0.176891         0.0048   
1  Kepler-974 b           0     NaN   0.140   4.194497         0.0390   
2   KOI-1843.02           0     NaN   0.071   6.356006         0.0520   
3    Kepler-9 b           0  0.2500   0.840  19.224180         0.1430   
4    Kepler-9 c           0  0.1700   0.820  39.031060         0.2290   

   eccentricity  periastron  longitude  ascendingnode  ...  lastupdate  \
0           NaN         NaN        NaN            NaN  ...    13/07/15   
1           NaN         NaN        NaN            NaN  ...    17/11/28   
2           NaN         NaN        NaN            NaN  ...         NaN   
3        0.0626         NaN        NaN            NaN  ...    15/12/03   
4        0.0684         NaN        NaN            NaN  ...    15/12/03   

   system_rightascension  system_declination system_distance  hoststar_mass  \
0      

In [78]:
# с помощью метода to_csv можно сохранить датафрейм в CSV-файл.
# online_exoplanets_df.to_csv('open_exoplanet_catalogue.csv', index=False) # index=False нужно, чтобы в файл не записывались индексы строк

## Обращение с отсутсвующими данными

Реальные датасеты часто имеют отсутствующие значения. Пустые места Pandas преставляет как `NaN` (Not a Number). Вам нужно научиться находить такие значение и взаимодействовать с ними, убирая такие строки совсем, или заполняя их чем-то существенным.

### Пример 10: обращаемся с NaN-ами

In [69]:
# Создадим НОВЫЙ датасет с отсутствующими данными
# импортируем из NumPy представление для nan
from numpy import nan
data_with_missing = {
    'Объект': ['Звезда A', 'Звезда Б', 'Галактика В', 'Звезда Г'],
    'Величина': [2.1, 3.5, nan, 4.8], 
    'Красное смещение': [0.001, 0.002, 0.015, nan]
}
missing_df = pd.DataFrame(data_with_missing)

print("Датафрейм с пустыми ячейками:")
print(missing_df)

Датафрейм с пустыми ячейками:
        Объект  Величина  Красное смещение
0     Звезда A       2.1             0.001
1     Звезда Б       3.5             0.002
2  Галактика В       NaN             0.015
3     Звезда Г       4.8               NaN


In [70]:
print("Проверка пустых значений с помощью isnull():")
print(missing_df.isnull()) # True будет там, где находятся NaN

print("\nЧисло пропущенных значений на колонку:")
print(missing_df.isnull().sum())

Проверка пустых значений с помощью isnull():
   Объект  Величина  Красное смещение
0   False     False             False
1   False     False             False
2   False      True             False
3   False     False              True

Число пропущенных значений на колонку:
Объект              0
Величина            1
Красное смещение    1
dtype: int64


In [71]:
# Вариант 1: убрать все строки, где есть хоть один NaN
cleaned_df_dropped_rows = missing_df.dropna()
print("Датафрейм после очистки строк с NaN:")
print(cleaned_df_dropped_rows)

Датафрейм после очистки строк с NaN:
     Объект  Величина  Красное смещение
0  Звезда A       2.1             0.001
1  Звезда Б       3.5             0.002


In [72]:
# Вариант 2: убрать все колонки, где есть хоть один NaN
cleaned_df_dropped_cols = missing_df.dropna(axis=1) # axis=1 означает колонки, axis=0 (по-умолчанию) - строки
print("Датафрейм без колонок с NaN:")
print(cleaned_df_dropped_cols)

Датафрейм без колонок с NaN:
        Объект
0     Звезда A
1     Звезда Б
2  Галактика В
3     Звезда Г


In [73]:
# Вариант 3: заменить NaN на что-то другое
filled_df = missing_df.fillna(value=0) # заменяем на 0
print("Датафрейм после замены NaN на 0:")
print(filled_df)

Датафрейм после замены NaN на 0:
        Объект  Величина  Красное смещение
0     Звезда A       2.1             0.001
1     Звезда Б       3.5             0.002
2  Галактика В       0.0             0.015
3     Звезда Г       4.8             0.000


In [76]:
# Чуть сложнее: заполним NaN в колонке звездных величин средней величиной из колонки
# mean() по-умолчанию игнорирует значения с NaN
mean_magnitude = missing_df['Величина'].mean()
filled_df_mean_mag = missing_df.fillna({'Величина': mean_magnitude}) # указываем словарь вида {'имя колонки': значение для замены}
print("Новый датафрейм:")
print(filled_df_mean_mag)

Новый датафрейм:
        Объект  Величина  Красное смещение
0     Звезда A  2.100000             0.001
1     Звезда Б  3.500000             0.002
2  Галактика В  3.466667             0.015
3     Звезда Г  4.800000               NaN


**Дополнительные материалы**

Шпаргалка по Pandas: <https://habr.com/ru/companies/ruvds/articles/494720/>

Getting started with Pandas: <https://pandas.pydata.org/docs/getting_started/index.html#getting-started>

Pandas user guide: <https://pandas.pydata.org/docs/user_guide/index.html#user-guide>

Groupby with Pandas: <https://realpython.com/pandas-groupby/>

Handling big data with pandas: <https://www.scaler.com/topics/pandas/handling-large-datasets-in-pandas/>

## Упражнения

1. Загрузите по URL из Примера 9 таблицу в датафрейм `exoplanets_df`. Если не получается загрузить по URL, считайте с помощью `read_csv` файл `open_exoplanet_catalogue.csv`.
2. Выведите из `exoplanets_df` только первые 5 значений колонки `mass`.
3. Выведите из `exoplanets_df` только строку под номером 2470.
4. Выведите радиус экзопланеты под номером 2471.
5. Выведите названия и расстояния до тех экзопланет, что находятся ближе 10 пк (`system_distance` должно быть меньше 10)
6. Выведите средний радиус экзопланеты из этого каталога в км (`radius` в каталоге дан в радиусах Юпитера, $R_J = 69 911$ км.)
7. Создайте новый датафрейм `clean_exoplanets_df`, который дублирует колонки предыдущего датафрейма `name`, `mass` и  `radius`, но содержит только строки со всеми тремя значениями (без NaN).
8. Создайте новую колонку `density` в `clean_exoplanets_df` со средней плотностью экзопланеты в г/см³ (`mass` приведены в массах Юпитера, $M_J = 1.9\cdot10^{30}$ г)
